In [2]:
import zipfile as zf
import os

os.chdir(my_features_path)
mfcc_files = zf.ZipFile("mfcc.zip", 'r')
mfcc_files.extractall('mfcc/')
mfcc_files.close()

In [6]:
import zipfile38 as zf38
mel_spectogram_files = zf38.ZipFile("mel_spectogram_features.zip", 'r')
mel_spectogram_files.extractall()
mel_spectogram_files.close()

In [4]:
gemaps_files =  zf38.ZipFile("gemaps_data.zip", 'r')
gemaps_files.extractall()
gemaps_files.close()

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
os.chdir('..')
os.listdir()

['best_model2',
 'deam-dataset',
 'valid_data.npz',
 'deam_data_fetch.py',
 'best_model',
 'deam_cnn_processing.ipynb',
 '.Trash-1000',
 'train_data.npz',
 'deam_cnn.py',
 'test_cnn_log.txt',
 'test_data.npz',
 'deam_cnn_model.ipynb',
 'DEAM_CNN-LSTM.ipynb',
 '.ipynb_checkpoints',
 'deam-dataset.zip',
 '__pycache__']

In [21]:
def fetch_mfcc(dir):
    files = os.listdir(dir)
    csv_files = list(filter(lambda f: '.csv' in f, files))
    mfcc_files = sorted(csv_files, key=lambda f: int(f[5:][:-4]))   # sort by audio index (remove 'mfcc' and '.csv')
    mfcc_data = np.empty(shape=(len(mfcc_files), 1920, 20), dtype=np.float32)
    for i, f in enumerate(mfcc_files):
        if '.csv' not in f: continue 
        p = os.path.join(dir, f)
        mfcc = np.loadtxt(p, delimiter=',')
        truncated_mfcc = mfcc[:, :1920]
        mfcc_data[i] = truncated_mfcc.T
    return mfcc_data

my_features_path = 'deam-dataset/my_features/'
mfcc_path = os.path.join(my_features_path, 'mfcc')
mfcc_data = fetch_mfcc(mfcc_path)
mfcc_data.shape

(1802, 1920, 20)

In [22]:
def fetch_mel_spectograms(dir):
    files = os.listdir(dir)
    csv_files = list(filter(lambda f: '.csv' in f, files))
    mel_files = sorted(csv_files, key=lambda f: int(f[len('mel_spectogram_'):][:-4]))   # sort by audio index (remove 'mel_spectogram_' and '.csv')
    mel_spec_data = np.empty(shape=(len(mel_files), 1920, 128, ), dtype=np.float32)

    for i, f in enumerate(mel_files):
        p = os.path.join(dir, f)
        mel_spec = np.loadtxt(p, delimiter=',', dtype=np.float32)
        truncated_mel_spec = mel_spec[:, :1920]
        mel_spec_data[i] = truncated_mel_spec.T
    return mel_spec_data

mel_path = os.path.join(my_features_path, 'mel_spectogram_features')
mel_spec_data = fetch_mel_spectograms(mel_path)
mel_spec_data.shape

(1802, 1920, 128)

In [23]:
np.save('mfcc_data', mfcc_data)
np.save('mel_spectogram_data', mel_spec_data)

In [24]:
# fetch labels
label_path = 'deam-dataset/DEAM_Annotations/annotations/annotations averaged per song/song_level/';
labels_1_2000 = pd.read_csv(os.path.join(label_path, 'static_annotations_averaged_songs_1_2000.csv'))
labels_2000_2058 = pd.read_csv(os.path.join(label_path,'static_annotations_averaged_songs_2000_2058.csv'))
labels = pd.concat([labels_1_2000, labels_2000_2058], ignore_index=True, sort=False)
labels = labels[labels_1_2000.columns]
labels.columns = labels.columns.str.replace(' ', '')

labels

,song_id,valence_mean,valence_std,arousal_mean,arousal_std
0,2,3.10,0.94,3.00,0.63
1,3,3.50,1.75,3.30,1.62
2,4,5.70,1.42,5.50,1.63
3,5,4.40,2.01,5.30,1.85
4,7,5.80,1.47,6.40,1.69
...,...,...,...,...,...
1797,2054,5.40,1.20,3.60,1.36
1798,2055,5.00,1.41,5.20,1.47
1799,2056,5.00,1.41,4.60,1.74
1800,2057,3.17,1.07,6.83,0.37


In [25]:
valence = labels['valence_mean'].to_numpy()
arousal = labels['arousal_mean'].to_numpy()
y = np.vstack([valence, arousal]).T
y.shape

(1802, 2)

In [26]:
concat_input = np.concatenate((mfcc_data, mel_spec_data), 2)
print('Concatenated input shape: ', concat_input.shape)

Concatenated input shape:  (1802, 1920, 148)
